# Lab work on Static Context Header Compression (SCHC)

This is part of the material for the lecture given at Polytech' Grenoble on IoT Networks by Quentin Lampin, PhD.
This labwork is both a crash-course on the Static Context Header Compression (SCHC) protocol and a hands-on session. 

The following is the hands-on session which covers: 

- the analysis of a traffic capture between a CoAP client and server using IPv6.
- the creation of Compression/Decompression rules for this traffic.
- the evaluation of rules built by students using usual Compression metrics.
- the study of the underlying trade-offs of building Compression/Decompression rules, including a discussion on rule generalization, rules count and memory. 


## Pre-requisites

Students are assumed to have installed Wireshark, a working 3.11 Python interpreter and the packages listed in `requirements.txt`


## Pre-flight checklist - 5 mins

**directions**

Run the following Jupyter notebook cell, report any error that might occur.

In [1]:
import pcapng
import microschc

## A look into the dataset - 15 mins

**directions**

Using Wireshark, open the dataset file: [`leshan-thermostat-readings.pcapng`](./dataset/leshan-thermostat-readings.pcapng) and answer the following questions.

- How many SCHC contexts?

- How many SCHC templates?

- What does each SCHC template correspond to?

- How many SCHC rules are necessary to compress all the packets of the dataset?


- For each template, list all constant fields

- What combination of Matching Operator and Compression/Decompression Action corresponds best to constant fields?

## Writing your own SCHC ruleset

microSCHC is a (micro)Python library that implements Compression and Decompression (C/D) functions of the SCHC protocol.

In the following part, you will write and evaluate SCHC C/D rules for a CoAP over UDP over IPv6 traffic, which is provided as a PCAPng capture file.

**directions**

Open the dataset and list packets contained within. (5mins)

**hints**

`tools.dataset` contains a helper function to open the dataset and list packets as `Buffer` items. 

In [2]:
from typing import List
from microschc.binary import Buffer
from tools.dataset import packets_list

packets: List[Buffer] = packets_list('./dataset/leshan-thermostat-readings.pcapng')
packets

[[600ff85f0020114020010db8000a0000000000000000000320010db8000a0000000000000000002090a01633002058215245145ed1596119622d16ffe816440840478ccccccccccd](576),
 [600ff85f001c114020010db8000a0000000000000000000320010db8000a0000000000000000002090a01633001cc36c5245145f3709611c613cfffb4031333333333333](544),
 [600ff85f0020114020010db8000a0000000000000000000320010db8000a0000000000000000002090a0163300209509524514602150611c622d16ffe81644084031333333333333](576),
 [600ff85f0020114020010db8000a0000000000000000000320010db8000a0000000000000000002090a016330020b81b52451461d159611c622d16ffe816440840472ccccccccccd](576),
 [600ff85f0020114020010db8000a0000000000000000000320010db8000a0000000000000000002090a016330020ae9d524514622150611f622d16ffe81644084031800000000000](576),
 [600ff85f0020114020010db8000a0000000000000000000320010db8000a0000000000000000002090a016330020f1af52451463d159611f622d16ffe8164408404759999999999a](576),
 [600ff85f0020114020010db8000a0000000000000000000320010db8000a0000000000000000002090


microSCHC provides parsers (`PacketParser`) for IPv6, UDP and CoAP protocols that provide descriptions of packets (`PacketDescriptor`).

A `PacketDescriptor` includes a list of `FieldDescriptor` which provide an overview of the packets fields, most notably field IDs and field lengths.

**Directions**: 

In the following cell, 

- instantiate a packet parser for the stack IPv6/UDP/CoAP and parse the packets of the dataset. (5mins)
- parse the dataset (5mins)

**hints**:

- Relevant microSCHC packets are `microschc.protocol.registry` and `microschc.parser`.
- `tools.packet` contains a helper function for printing packet descriptors.


In [3]:
from microschc.protocol.registry import Stack, factory
from microschc.parser import PacketParser
from microschc.rfc8724 import PacketDescriptor

from tools.packet import packet_descriptor_as_asciitable

packet_parser: PacketParser = factory(stack_id=Stack.IPV6_UDP_COAP)

packet_descriptors: List[PacketDescriptor] = [packet_parser.parse(packet) for packet in packets]
print(packet_descriptor_as_asciitable(packet_descriptors[0]))

+--------------------------+-----------------------------------------+
|         field ID         |                  value                  |
+==========================+=========================================+
| IPv6:Version             | [----0110 ](4)                          |
+--------------------------+-----------------------------------------+
| IPv6:Traffic Class       | [00000000](8)                           |
+--------------------------+-----------------------------------------+
| IPv6:Flow Label          | [0ff85f](20)                            |
+--------------------------+-----------------------------------------+
| IPv6:Payload Length      | [00000000 00100000](16)                 |
+--------------------------+-----------------------------------------+
| IPv6:Next Header         | [00010001](8)                           |
+--------------------------+-----------------------------------------+
| IPv6:Hop Limit           | [01000000](8)                           |
+-----

C/D rules are composed of Rule Field Descriptors. To be eligible for compression, every field of the parsed packet has to be matched to a field descriptor of the C/D rule.
It is therefore sensible to group packets with the same structure (template) before addressing them with C/D rules.

**directions**

- List packets templates of the dataset (5mins)
- print their structure (5mins)

**hints**

- `find_templates` in `tools.template` is your friend ;)
- `template_as_asciitable` in `tools.template` can be useful to print the templates.

In [4]:
from tools.template import find_templates, Template, template_as_asciitable

templates: List[Template] = find_templates(packets, parser=packet_parser)
print(f' found {len(templates)} templates')
for i, template in enumerate(templates):
    print(f'template {i}: {len(template.packets)} packets')
    print(template_as_asciitable(template))

 found 6 templates
template 0: 8543 packets
+--------------------------+--------+
|         field ID         | length |
+==========================+========+
| IPv6:Version             | 4      |
+--------------------------+--------+
| IPv6:Traffic Class       | 8      |
+--------------------------+--------+
| IPv6:Flow Label          | 20     |
+--------------------------+--------+
| IPv6:Payload Length      | 16     |
+--------------------------+--------+
| IPv6:Next Header         | 8      |
+--------------------------+--------+
| IPv6:Hop Limit           | 8      |
+--------------------------+--------+
| IPv6:Source Address      | 128    |
+--------------------------+--------+
| IPv6:Destination Address | 128    |
+--------------------------+--------+
| UDP:Source Port          | 16     |
+--------------------------+--------+
| UDP:Destination Port     | 16     |
+--------------------------+--------+
| UDP:Length               | 16     |
+--------------------------+--------+
| UDP:

Packets within the same template share the same structure, i.e. same fields in the same order, but one field may have different values from one packet to another.
As a first step, one can write a rule that compresses all constant fields for a given template. 


**directions**

- find all constant fields of the first template (#0) (10 mins)
- print them (42 sec)
- find all variable fields fo the first template (2mins)
- print them (42s)


**hints**

- `enumerate` is a nifty function that enumerate items of a list, providing the index of each element
- a `set` is a practical data structure to store unique values
- `filter` provides an elegant way to filter elements of an iterable 
- `fields_as_asciitable` in `tools.field` can be used to print fields

In [5]:
from typing import Set, Tuple
from tools.field import fields_as_asciitable

first_template: Template = templates[0]

field_values: List[Tuple[int, str, Set[Buffer]]] = [(i, field.id.value, set()) for i, field in enumerate(first_template.fields)]

for packet in first_template.packets:
    for i, field in enumerate(packet.fields):
        field_values[i][2].add(field.value)

constant_fields: List[Tuple[int, str, Set[Buffer]]] = list(filter(lambda t: len(t[2]) == 1, field_values))
variable_fields: List[Tuple[int, str, Set[Buffer]]] = list(filter(lambda t: len(t[2]) > 1, field_values))

print('constant fields:')
print(fields_as_asciitable(constant_fields))

print ('variable fields:')
print(fields_as_asciitable(variable_fields))



constant fields:
+------------------------------+-------------------------------------------+
|           field ID           |                   value                   |
+==============================+===========================================+
| 0 - IPv6:Version             | {[----0110 ](4)}                          |
+------------------------------+-------------------------------------------+
| 1 - IPv6:Traffic Class       | {[00000000](8)}                           |
+------------------------------+-------------------------------------------+
| 2 - IPv6:Flow Label          | {[0ff85f](20)}                            |
+------------------------------+-------------------------------------------+
| 4 - IPv6:Next Header         | {[00010001](8)}                           |
+------------------------------+-------------------------------------------+
| 5 - IPv6:Hop Limit           | {[01000000](8)}                           |
+------------------------------+---------------------------

A SCHC C/D Rule is made of a list of field descriptors. a field descriptor has 4 main components: the Field ID (FID), the Matching Operator (MO), the Compression Decompression Action (CDA) and the Target Value (TV).

The FID corresponds to the Field Id of the field to compress, the MO indicates which packets can be compressed, the CDA tells how packets are compressed and the TV provides parameters to the CDA.

According to your knowledge, which MO/CDA couple should be used for constant fields?

Using microschc, create a rule matching the packet structure of template #0 that compresses every constant field.

** directions **

- import the fields definitions for IPv6, UDP and CoAP

** hints **

- protocol definitions are located in the `microschc.protocol` package
- rule field descriptor and rule descriptor are defined `microschc.rfc8724`

In [6]:
from microschc.protocol.ipv6 import IPv6Fields
from microschc.protocol.udp import UDPFields
from microschc.protocol.coap import CoAPFields
from microschc.rfc8724 import RuleFieldDescriptor, RuleDescriptor
from microschc.rfc8724 import MatchingOperator as MO
from microschc.rfc8724 import CompressionDecompressionAction as CDA
from microschc.rfc8724 import DirectionIndicator as DI
from microschc.binary.buffer import Padding

from tools.rule import rule_descriptor_as_asciitable

constant_fields_indices: List[int] = [t[0] for t in constant_fields]
variable_fields_indices: List[int] = [t[0] for t in variable_fields]

rule_0_field_descriptors = []

for i, field_tuple in enumerate(field_values):
    if i in constant_fields_indices:
        field_index, field_id, value_set = field_tuple
        field_value: Buffer = list(value_set)[0]
        field_length: int = field_value.length

        rule_field_descriptor: RuleFieldDescriptor = RuleFieldDescriptor(
            id=field_id, length=field_length, position=0, direction=DI.BIDIRECTIONAL, 
            matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT,
            target_value=field_value
        )
    else:
        field_index, field_id, value_set = field_tuple
        value_lengths: Set[int] = {value.length for value in value_set}
        field_length: int = 0 if len(value_lengths) > 0 else list(value_lengths)[0]
        
        rule_field_descriptor: RuleFieldDescriptor = RuleFieldDescriptor(
            id=field_id, length=field_length, position=0, direction=DI.BIDIRECTIONAL, 
            matching_operator=MO.IGNORE, compression_decompression_action=CDA.VALUE_SENT,
            target_value=Buffer(content=b'', length=0, padding=Padding.LEFT)
        )
    rule_0_field_descriptors.append(rule_field_descriptor)



rule_0_descriptor: RuleDescriptor = RuleDescriptor(id=Buffer(content=b'\x00', length=4), field_descriptors=rule_0_field_descriptors)
print(rule_descriptor_as_asciitable(rule_0_descriptor))


+--------------------------+-----+----------------------------------+--------+-----------------------------------------+
|           FID            | LEN |                FD                | MO/CDA |                   TV                    |
+==========================+=====+==================================+========+=========================================+
| IPv6:Version             | 4   | DirectionIndicator.BIDIRECTIONAL | eq/ns  | [----0110 ](4)                          |
+--------------------------+-----+----------------------------------+--------+-----------------------------------------+
| IPv6:Traffic Class       | 8   | DirectionIndicator.BIDIRECTIONAL | eq/ns  | [00000000](8)                           |
+--------------------------+-----+----------------------------------+--------+-----------------------------------------+
| IPv6:Flow Label          | 20  | DirectionIndicator.BIDIRECTIONAL | eq/ns  | [0ff85f](20)                            |
+--------------------------+----

In [7]:
# manual solution
rule_0_ipv6_version: RuleFieldDescriptor = RuleFieldDescriptor(
    id=IPv6Fields.VERSION, length=4, position=0, direction=DI.BIDIRECTIONAL, 
    matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT,
    target_value=Buffer(content=b'\x06', length=4),
)

rule_0_ipv6_traffic_class = RuleFieldDescriptor(
    id=IPv6Fields.TRAFFIC_CLASS, length=8, position=0, direction=DI.BIDIRECTIONAL, 
    matching_operator=MO.IGNORE, compression_decompression_action=CDA.NOT_SENT,
    target_value=Buffer(content=b'\x00', length=8), 
)

rule_0_ipv6_flow_label = RuleFieldDescriptor(
    id=IPv6Fields.FLOW_LABEL, length=20, position=0, direction=DI.BIDIRECTIONAL, 
    matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT,
    target_value=Buffer(content=b'\x0f\xf8\x5f', length=20, padding=Padding.LEFT),
)

rule_0_ipv6_payload_length = RuleFieldDescriptor(
    id=IPv6Fields.PAYLOAD_LENGTH, length=16, position=0, direction=DI.BIDIRECTIONAL, 
    matching_operator=MO.IGNORE, compression_decompression_action=CDA.VALUE_SENT,
    target_value=Buffer(content=b'', length=0, padding=Padding.LEFT), 
)

rule_0_ipv6_next_header = RuleFieldDescriptor(
    id=IPv6Fields.NEXT_HEADER, length=8, position=0, direction=DI.BIDIRECTIONAL,
    matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT,
    target_value=Buffer(content=b'\x11', length=8)
)

rule_0_ipv6_hop_limit = RuleFieldDescriptor(
    id=IPv6Fields.HOP_LIMIT, length=8, position=0, direction=DI.BIDIRECTIONAL,
    matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT,
    target_value=Buffer(content=b'\x40', length=8),
)

rule_0_ipv6_source_address = RuleFieldDescriptor(
    id=IPv6Fields.SRC_ADDRESS, length=128, position=0, direction=DI.BIDIRECTIONAL,
    matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT,
    target_value=Buffer(content=b"\x20\x01\x0d\xb8\x00\x0a\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03", length=128), 
)

rule_0_ipv6_destination_address = RuleFieldDescriptor(
    id=IPv6Fields.DST_ADDRESS, length=128, position=0, direction=DI.BIDIRECTIONAL,
    matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT,
    target_value=Buffer(content=b"\x20\x01\x0d\xb8\x00\x0a\x00\x00\x00\x00\x00\x00\x00\x00\x00\x20", length=128), 
)

rule_0_udp_source_port = RuleFieldDescriptor(
    id=UDPFields.SOURCE_PORT, length=16, position=0, direction=DI.BIDIRECTIONAL,
    matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT,
    target_value=Buffer(content=b"\x90\xa0", length=16), 
)

rule_0_udp_destination_port = RuleFieldDescriptor(
    id=UDPFields.DESTINATION_PORT, length=16, position=0, direction=DI.BIDIRECTIONAL,
    matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT,
    target_value=Buffer(content=b"\x16\x33", length=16), 
)

rule_0_udp_length = RuleFieldDescriptor(
    id=UDPFields.LENGTH, length=16, position=0, direction=DI.BIDIRECTIONAL,
    matching_operator=MO.IGNORE, compression_decompression_action=CDA.VALUE_SENT,
    target_value=Buffer(content=b"", length=0)
)

rule_0_udp_checksum = RuleFieldDescriptor(
    id=UDPFields.CHECKSUM, length=16, position=0, direction=DI.BIDIRECTIONAL,
    matching_operator=MO.IGNORE, compression_decompression_action=CDA.VALUE_SENT,
    target_value=Buffer(content=b"", length=0)
)

rule_0_coap_version = RuleFieldDescriptor(
    id=CoAPFields.VERSION, length=2, position=0, direction=DI.BIDIRECTIONAL,
    matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT,
    target_value=Buffer(content=b"\x01", length=2)
)

rule_0_coap_type = RuleFieldDescriptor(
    id=CoAPFields.TYPE, length=2, position=0, direction=DI.BIDIRECTIONAL,
    matching_operator=MO.MSB, compression_decompression_action=CDA.LSB,
    target_value=Buffer(content=b"\x00", length=1, padding=Padding.RIGHT)
)

rule_0_coap_token_length = RuleFieldDescriptor(
    id=CoAPFields.TOKEN_LENGTH, length=4, position=0, direction=DI.BIDIRECTIONAL,
    matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT,
    target_value=Buffer(content=b"\x02", length=4)
)    

rule_0_coap_code = RuleFieldDescriptor(
    id=CoAPFields.CODE, length=8, position=0, direction=DI.BIDIRECTIONAL,
    matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT,
    target_value=Buffer(content=b"\x45", length=8)
)  

rule_0_coap_message_id = RuleFieldDescriptor(
    id=CoAPFields.MESSAGE_ID, length=16, position=0, direction=DI.BIDIRECTIONAL,
    matching_operator=MO.IGNORE, compression_decompression_action=CDA.VALUE_SENT,
    target_value=Buffer(content=b"", length=0)
)  

rule_0_coap_token = RuleFieldDescriptor(
    id=CoAPFields.TOKEN, length=16, position=0, direction=DI.BIDIRECTIONAL,
    matching_operator=MO.IGNORE, compression_decompression_action=CDA.VALUE_SENT,
    target_value=Buffer(content=b"", length=0)
) 

rule_0_coap_option_delta_1 = RuleFieldDescriptor(
    id=CoAPFields.OPTION_DELTA, length=4, position=0, direction=DI.BIDIRECTIONAL,
    matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT,
    target_value=Buffer(content=b"\x06", length=4, padding=Padding.LEFT)
) 

rule_0_coap_option_length_1 = RuleFieldDescriptor(
    id=CoAPFields.OPTION_LENGTH, length=4, position=0, direction=DI.BIDIRECTIONAL,
    matching_operator=MO.MSB, compression_decompression_action=CDA.LSB,
    target_value=Buffer(content=b"\x00", length=2, padding=Padding.RIGHT),
) 

rule_0_coap_option_value_1 = RuleFieldDescriptor(
    id=CoAPFields.OPTION_VALUE, length=0, position=0, direction=DI.BIDIRECTIONAL,
    matching_operator=MO.IGNORE, compression_decompression_action=CDA.VALUE_SENT,
    target_value=Buffer(content=b"", length=0)
)     

rule_0_coap_option_delta_2 = RuleFieldDescriptor(
    id=CoAPFields.OPTION_DELTA, length=4, position=0, direction=DI.BIDIRECTIONAL,
    matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT,
    target_value=Buffer(content=b"\x06", length=4, padding=Padding.LEFT)
) 

rule_0_coap_option_length_2 = RuleFieldDescriptor(
    id=CoAPFields.OPTION_LENGTH, length=4, position=0, direction=DI.BIDIRECTIONAL,
    target_value=Buffer(content=b"\x02", length=4, padding=Padding.LEFT),
    matching_operator=MO.EQUAL,
    compression_decompression_action=CDA.NOT_SENT
) 

rule_0_coap_option_value_2 = RuleFieldDescriptor(
    id=CoAPFields.OPTION_VALUE, length=16, position=0, direction=DI.BIDIRECTIONAL,
    matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT,
    target_value=Buffer(content=b"\x2d\x16", length=16)
)   

rule_0_coap_payload_marker = RuleFieldDescriptor(
    id=CoAPFields.PAYLOAD_MARKER, length=8, position=0, direction=DI.BIDIRECTIONAL,
    matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT,
    target_value=Buffer(content=b"\xff", length=8)
)   


rule_0_field_descriptors: List[RuleFieldDescriptor] = [
    rule_0_ipv6_version,
    rule_0_ipv6_traffic_class,
    rule_0_ipv6_flow_label,
    rule_0_ipv6_payload_length,
    rule_0_ipv6_next_header, 
    rule_0_ipv6_hop_limit,
    rule_0_ipv6_source_address,
    rule_0_ipv6_destination_address,
    rule_0_udp_source_port,
    rule_0_udp_destination_port,
    rule_0_udp_length,
    rule_0_udp_checksum,
    rule_0_coap_version,
    rule_0_coap_type,
    rule_0_coap_token_length,
    rule_0_coap_code, 
    rule_0_coap_message_id,
    rule_0_coap_token,
    rule_0_coap_option_delta_1,
    rule_0_coap_option_length_1,
    rule_0_coap_option_value_1,
    rule_0_coap_option_delta_2,
    rule_0_coap_option_length_2,
    rule_0_coap_option_value_2,
    rule_0_coap_payload_marker
]

rule_0_descriptor: RuleDescriptor = RuleDescriptor(id=Buffer(content=b'\x00', length=4), field_descriptors=rule_0_field_descriptors)
print(rule_descriptor_as_asciitable(rule_0_descriptor))

+----------------------------+-----+----------------------------------+--------+-----------------------------------------+
|            FID             | LEN |                FD                | MO/CDA |                   TV                    |
+============================+=====+==================================+========+=========================================+
| IPv6Fields.VERSION         | 4   | DirectionIndicator.BIDIRECTIONAL | eq/ns  | [----0110 ](4)                          |
+----------------------------+-----+----------------------------------+--------+-----------------------------------------+
| IPv6Fields.TRAFFIC_CLASS   | 8   | DirectionIndicator.BIDIRECTIONAL | ig/ns  | [00000000](8)                           |
+----------------------------+-----+----------------------------------+--------+-----------------------------------------+
| IPv6Fields.FLOW_LABEL      | 20  | DirectionIndicator.BIDIRECTIONAL | eq/ns  | [0ff85f](20)                            |
+---------------

To produce a minimum SCHC Context, we need to add the no-compression rule, a rule that tells the residue is not compressed.

A no-compression rule is only defined by its nature and a Rule ID.

**directions**

create a SCHC context with your previous rule and the no-compression rule.

**hints**

- a no-compression rule is defined using:
```
no_compression_rule_descriptor: RuleDescriptor = RuleDescriptor(id=Buffer(content=b'\xXX', length=Y), nature=RuleNature.NO_COMPRESSION)
```

- the SCHC context is defined in the  `microschc.rfc8724extras` package



In [8]:
from microschc.rfc8724 import RuleNature
from microschc.rfc8724extras import Context

#No compression rule (set appropriate ID and id length)
no_compression_rule_descriptor: RuleDescriptor = RuleDescriptor(id=Buffer(content=b'\x01', length=4), nature=RuleNature.NO_COMPRESSION)

rule_descriptors = [
    rule_0_descriptor, 
    no_compression_rule_descriptor
]

schc_context: Context = Context(
    id='PolytechGrenoble',
    description='context used in the labwork',
    interface_id='none',
    ruleset=rule_descriptors,
    parser_id=Stack.IPV6_UDP_COAP
)

print (schc_context)


id:PolytechGrenoble description:context used in the labwork interface_id: none parser_id: Stack.IPV6_UDP_COAP rules: 2


## Evaluating your SCHC rulesset

Now is time to evaluate your first ruleset. 

**directions**

- evaluate your ruleset against the dataset
- print Compression Factor (CF) statistics for your ruleset

**hints**

- the `tools.context` package contains a function to evaluate the context, in case you're running out of time.



In [10]:
from microschc.manager import ContextManager, MatchStrategy
from tools.context import ContextStatistics, evaluate, context_statistics_as_ascii_table



statistics: ContextStatistics = evaluate(context=schc_context, templates=templates, packets=packets, match_strategy=MatchStrategy.BEST)

print(context_statistics_as_ascii_table(statistics))


+-----+----------+-------+-------+---------+--------------+----------------+---------+---------+
| ID  | template | hits  |  CF   |  CFOC   | compressible | incompressible |  total  | residue |
+=====+==========+=======+=======+=========+==============+================+=========+=========+
| 0   | 0        | 5900  | 2.772 | 4.257   | 2877976      | 566400         | 3444376 | 1242452 |
+-----+----------+-------+-------+---------+--------------+----------------+---------+---------+
| 1   | 6        | 4100  | 0.992 | 0.992   | 1954280      | 171504         | 2125784 | 2142184 |
+-----+----------+-------+-------+---------+--------------+----------------+---------+---------+
| all | 10000    | 1.646 | 1.826 | 4832256 | 737904       | 5570160        | 3384636 | N/A     |
+-----+----------+-------+-------+---------+--------------+----------------+---------+---------+


## Going Further with SCHC

From now on, you can choose between the following instructions:

- augment your ruleset to address packets from the other templates
- improve your compression rule to address fields with few values (Match-Mapping)
- investigate MSB/LSB MO/CDA application
